In [1]:
import pandas as pd
import altair as alt
from pathlib import Path
import polars as pl
import tools as nx
import yaml
#from utils import load_config


## Force reload tools

In [2]:
import importlib
importlib.reload(nx)
print(nx.__file__)
print(dir(nx))

/Users/osaa/Documents/code/nexus-package/tools/__init__.py
['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'datamap', 'filter_pivot']


## Setup

In [3]:
# 1. Load configuration
with open("config.yaml") as f:
    config = yaml.safe_load(f)

# 2. Project Paths
PROJECT_ROOT = Path.cwd()
DATA_PATH    = PROJECT_ROOT / config["data"]["nexus_path"]

# 3. Query settings
INDEX_COLS      = config["query"]["index_columns"]
SOURCE_META     = config["query"]["source_metadata_columns"]
IND_META        = config["query"]["indicator_metadata_columns"]
COUNTRY_CLASSES = config["query"]["country_classification_columns"]

# 4. Altair transformer
alt.data_transformers.enable('default', max_rows=None)

# 5. Polars table display
pl.Config.set_tbl_rows(-1)  # Show all rows
pl.Config.set_tbl_cols(-1)  # Show all columns

polars.config.Config

In [4]:
# 1️⃣ Lazy‑load the Parquet into a LazyFrame
nexus = (
    pl.scan_parquet(DATA_PATH)
    .rename({"country_or_area": "country"})
    .with_columns(
        pl.when(pl.col("high_income") == "High income")
        .then(pl.lit("High Income"))
        .when(pl.col("upper_middle_income") == "Upper middle income")
        .then(pl.lit("Upper Middle Income"))
        .when(pl.col("lower_middle_income") == "Lower middle income")
        .then(pl.lit("Lower Middle Income"))
        .when(pl.col("low_income") == "Low income")
        .then(pl.lit("Low Income"))
        .otherwise(None)
        .cast(pl.Categorical)
        .alias("income_level")
    )
)

Testing the income_level classification

In [5]:
# Check income_level for a sample of countries
test_countries = ["United States", "China", "Nigeria", "Bangladesh", "Brazil"]

income_check = (
    nexus
    .filter(pl.col("country").is_in(test_countries))
    .select(["country", "income_level", "high_income", "upper_middle_income", "lower_middle_income", "low_income"])
    .unique(["country"])
    .collect()
)

print(income_check)

# Also check the distribution of income levels
income_distribution = (
    nexus
    .select("income_level")
    .group_by("income_level")
    .agg(pl.len().alias("count"))
    .collect()
)

print("\nIncome Level Distribution:")
print(income_distribution.sort("count", descending=True))

shape: (4, 6)
┌────────────┬──────────────┬─────────────┬─────────────────────┬─────────────────────┬────────────┐
│ country    ┆ income_level ┆ high_income ┆ upper_middle_income ┆ lower_middle_income ┆ low_income │
│ ---        ┆ ---          ┆ ---         ┆ ---                 ┆ ---                 ┆ ---        │
│ str        ┆ cat          ┆ str         ┆ str                 ┆ str                 ┆ str        │
╞════════════╪══════════════╪═════════════╪═════════════════════╪═════════════════════╪════════════╡
│ China      ┆ Upper Middle ┆ null        ┆ Upper middle income ┆ null                ┆ null       │
│            ┆ Income       ┆             ┆                     ┆                     ┆            │
│ Nigeria    ┆ Lower Middle ┆ null        ┆ null                ┆ Lower middle income ┆ null       │
│            ┆ Income       ┆             ┆                     ┆                     ┆            │
│ Brazil     ┆ Upper Middle ┆ null        ┆ Upper middle income ┆ null       

In [6]:
# Count distinct countries with null income_level
null_income_countries = (
   nexus
   .filter(pl.col("income_level").is_null())
   .select(pl.col("country").n_unique())
   .collect()
   .item()
)

print(f"Number of distinct countries with null income_level: {null_income_countries}")

# Optionally, see which countries have null income_level
countries_with_null_income = (
   nexus
   .filter(pl.col("income_level").is_null())
   .select("country")
   .unique()
   .collect()
   .sort("country")
)

print(f"\nCountries with null income_level:")
print(countries_with_null_income)
# countries_with_null_income.write_csv("countries_with_null_income.csv")

Number of distinct countries with null income_level: 27

Countries with null income_level:
shape: (27, 1)
┌─────────────────────────────────┐
│ country                         │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ null                            │
│ Anguilla                        │
│ Bonaire, Sint Eustatius and Sa… │
│ Bouvet Island                   │
│ Christmas Island                │
│ Cocos (Keeling) Islands         │
│ Cook Islands                    │
│ Falkland Islands (Malvinas)     │
│ French Guiana                   │
│ Guadeloupe                      │
│ Guernsey                        │
│ Heard Island and McDonald Isla… │
│ Holy See                        │
│ Jersey                          │
│ Martinique                      │
│ Montserrat                      │
│ Namibia                         │
│ Niue                            │
│ Norfolk Island                  │
│ Pitcairn                    

# NEXUS OBT

In [15]:
# 2️⃣ “.info()” → inspect the schema without fetching all the data:
schema = nexus.collect_schema()
print(schema)  

Schema({'year': Int64, 'value': Float64, 'source': String, 'indicator_code': String, 'indicator_label': String, 'database': String, 'collection': String, 'value_meta': String, 'global_code': Float64, 'global_name': String, 'region_code': Float64, 'region_name': String, 'sub_region_code': Float64, 'sub_region_name': String, 'intermediate_region_code': Float64, 'intermediate_region_name': String, 'country': String, 'm49': Float64, 'iso2': String, 'iso3': String, 'least_developed_countries_ldc': String, 'land_locked_developing_countries_lldc': String, 'small_island_developing_states_sids': String, 'arab_states': String, 'fragile_and_conflict_affected_situations': String, 'hipc': String, 'high_income': String, 'low_income': String, 'lower_middle_income': String, 'oecd_member': String, 'small_state': String, 'ldc_transit_countries': String, 'upper_middle_income': String, 'oil_exporting_countries': String})


Or the full collect shabang

In [16]:
df = nexus.collect()      # run the query
print(df.shape)           # (n_rows, n_cols)
print(df.dtypes)          # list of dtypes
df.describe()             # summary stats for each column

(616409, 34)
[Int64, Float64, String, String, String, String, String, String, Float64, String, Float64, String, Float64, String, Float64, String, String, Float64, String, String, String, String, String, String, String, String, String, String, String, String, String, String, String, String]


statistic,year,value,source,indicator_code,indicator_label,database,collection,value_meta,global_code,global_name,region_code,region_name,sub_region_code,sub_region_name,intermediate_region_code,intermediate_region_name,country,m49,iso2,iso3,least_developed_countries_ldc,land_locked_developing_countries_lldc,small_island_developing_states_sids,arab_states,fragile_and_conflict_affected_situations,hipc,high_income,low_income,lower_middle_income,oecd_member,small_state,ldc_transit_countries,upper_middle_income,oil_exporting_countries
str,f64,f64,str,str,str,str,str,str,f64,str,f64,str,f64,str,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""",616409.0,582793.0,"""616409""","""616409""","""616409""","""616409""","""616409""","""33928""",603777.0,"""603777""",603777.0,"""603777""",603777.0,"""603777""",256549.0,"""273893""","""603777""",603777.0,"""600615""","""603777""","""126272""","""98110""","""101839""","""48563""","""94083""","""119838""","""193793""","""73566""","""169523""","""145856""","""128530""","""108092""","""159251""","""213860"""
"""null_count""",0.0,33616.0,"""0""","""0""","""0""","""0""","""0""","""582481""",12632.0,"""12632""",12632.0,"""12632""",12632.0,"""12632""",359860.0,"""342516""","""12632""",12632.0,"""15794""","""12632""","""490137""","""518299""","""514570""","""567846""","""522326""","""496571""","""422616""","""542843""","""446886""","""470553""","""487879""","""508317""","""457158""","""402549"""
"""mean""",2006.26877,2.0173e12,null,null,null,null,null,null,1.0,null,73.835462,null,175.96159,null,14.791159,null,null,428.680016,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",11.425231,1.4523e14,null,null,null,null,null,null,0.0,null,68.703664,null,130.692135,null,7.140698,null,null,250.879051,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""",1960.0,-4.4363e11,"""GFI""","""Academic qualifications (No. o…","""Academic qualifications (No. o…","""Collecting Taxes Database (CTD…","""Audit and verification""","""..""",1.0,"""World""",2.0,"""Africa""",15.0,"""Australia and New Zealand""",5.0,"""Caribbean""","""Afghanistan""",4.0,"""AD""","""ABW""","""Least Developed Countries (LDC…","""Land Locked Developing Countri…","""Small Island Developing States…","""Arab states""","""Fragile and Conflict-Affected …","""HIPC""","""High income""","""Low income""","""Lower middle income""","""OECD member""","""Small_state""","""Transit countries""","""Upper middle income""","""Oil Exporting"""
"""25%""",1999.0,1.0,null,null,null,null,null,null,1.0,null,2.0,null,39.0,null,11.0,null,null,208.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""50%""",2008.0,6.516388,null,null,null,null,null,null,1.0,null,19.0,null,154.0,null,14.0,null,null,426.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""75%""",2016.0,45.0,null,null,null,null,null,null,1.0,null,142.0,null,202.0,null,17.0,null,null,642.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""max""",2024.0,3.9400e16,"""World Bank""","""va""","""sotj23_loss_corp_musd""","""wgidataset.xlsx""","""WGI""","""o""",1.0,"""World""",150.0,"""Oceania""",419.0,"""Western Europe""",29.0,"""Western Africa""","""Zimbabwe""",894.0,"""ZW""","""ZWE""","""Least Developed Countries (LDC…","""Land Locked Developing Countri…","""Small Island Developing States…","""Arab states""","""Fragile and Conflict-Affected …","""HIPC""","""High income""","""Low income""","""Lower middle income""","""OECD member""","""Small_state""","""Transit countries""","""Upper middle income""","""Oil Exporting"""


## Nexus metadata

In [9]:
# Get distinct counts for all metadata columns
all_metadata_cols = SOURCE_META + IND_META + COUNTRY_CLASSES

distinct_counts = (
   nexus
   .select([pl.col(c).n_unique().alias(c) for c in all_metadata_cols])
   .collect()
)

# Display as a transposed view
distinct_counts.transpose(include_header=True, header_name="column", column_names=["distinct_count"])

column,distinct_count
str,u32
"""source""",6
"""database""",12
"""collection""",32
"""indicator_code""",407
"""value_meta""",7
…,…
"""land_locked_developing_countri…",2
"""small_island_developing_states…",2
"""income_level""",5


## `datamap` tool

In [8]:
nx.datamap

<function tools.datamap.datamap(df: polars.lazyframe.frame.LazyFrame, source_metadata_columns: List[str], indicator_metadata_columns: Optional[List[str]] = None) -> polars.dataframe.frame.DataFrame>

In [7]:
nx.datamap(
    nexus,
    source_metadata_columns=SOURCE_META
)

source,database,collection,count
str,str,str,u32
"""GFI""","""gfi trade mispricing.xlsx""","""Table A""",1195
"""GFI""","""gfi trade mispricing.xlsx""","""Table C""",1185
"""GFI""","""gfi trade mispricing.xlsx""","""Table E""",1199
"""GFI""","""gfi trade mispricing.xlsx""","""Table G""",1189
"""ISORA""","""IMF ISORA.xlsx""","""Electronic filing rates by tax…",1284
"""ISORA""","""IMF ISORA.xlsx""","""On-time filing rates by tax ty…",1725
"""ISORA""","""IMF ISORA.xlsx""","""Percentage inactive taxpayers …",2027
"""ISORA""","""IMF ISORA.xlsx""","""Proportion of returns by ch_0""",2162
"""ISORA""","""IMF ISORA.xlsx""","""Proportion of returns by ch_1""",2027


## `filter_pivot` tool

Using `filter_pivot` with config values

In [8]:
nx.filter_pivot(
    nexus.filter(pl.col("source") == "ISORA"),
    index_cols=INDEX_COLS,
    ind='label'  # or 'code' to pivot on indicator_code
)

country,year,Capital expenditure - Derived,Operating expenditure - Derived,Salary expenditure - Derived,Information and communications technology expenditure - Derived,FTEs by function of the tax administration-Other functions,Percentage of staff working on headquarter functions,Total tax administration FTEs - Derived,"FTEs by function of the tax administration-Registration, taxpayer services, returns and payment processing",FTEs by function of the tax administration-Enforced debt collection and related functions,"FTEs by function of the tax administration-Audit, investigation and other verification",Operational ICT solutions of the administration are…-Custom built,Operational ICT solutions of the administration are…-On premises commercial off the shelf (COTS),"Operational ICT solutions of the administration are…-Software-as-a-Service (SaaS, i.e. cloud based)",Staff strength levels -No. at end of FY,Staff strength levels -No. at start of FY,Staff strength levels -Departures in FY,Staff strength levels -Recruitments in FY,Academic qualifications (No. of staff at the end of FY)-Masters degree (or above),Academic qualifications (No. of staff at the end of FY)-Bachelors degree,Age distribution (No. of staff at the end of FY)-35-44 years,Age distribution (No. of staff at the end of FY)-45-54 years,Age distribution (No. of staff at the end of FY)-25-34 years,Age distribution (No. of staff at the end of FY)-Under 25 years,Age distribution (No. of staff at the end of FY)-Over 64 years,Age distribution (No. of staff at the end of FY)-55-64 years,Length of service (No. of staff at the end of FY)-Over 19 years,Length of service (No. of staff at the end of FY)-Under 5 years,Length of service (No. of staff at the end of FY)-5-9 years,Length of service (No. of staff at the end of FY)-10-19 years,Gender distribution (No. of staff at the end of FY)-Executives only-Female,Gender distribution (No. of staff at the end of FY)-Executives only-Other,Gender distribution (No. of staff at the end of FY)-All staff-Male,Gender distribution (No. of staff at the end of FY)-All staff-Female,Gender distribution (No. of staff at the end of FY)-Executives only-Male,Gender distribution (No. of staff at the end of FY)-All staff-Other,…,Taxpayers must first pursue internal review where an internal review is permissible,Mechanisms available for taxpayers to challenge assessments-Independent review by external body,FTEs in LTO/P as percentage of FTEs,Percentage of net revenue administered under LTO/P in relation to total net revenue collected by the tax administration,"FTEs on audit, investigation and other verification function in the LTO/P as percentage of total FTEs in LTO/P",Corporate taxpayers managed through LTO/P as percentage of active corporate taxpayers,Total value of additional assessments raised through LTO/P as percentage of total value of additional assessments raised from audits,Corporate taxpayers per FTE in LTO/P,Active taxpayers on PIT register as percentage of Labor Force,Active taxpayers on PIT register as percentage of Population,On PIT register,On CIT register,On PAYE register,On VAT register,On Excise register,On-time filing rate % - Corporate income tax,On-time filing rate % - Personal income tax,On-time filing rate % - Value added tax,On-time filing rate % - Employers that withhold tax from employees,Percent CIT returns e-filed,Percent PIT returns e-filed,Percent VAT returns e-filed,Paper returns-CIT,Electronic - fully pre-filled deemed-CIT,Electronic -not pre-filled or partially pre-filled-CIT,Electronic - fully pre-filled confirmation required-CIT,Other-CIT,Paper returns-PIT,Electronic - fully pre-filled deemed-PIT,Electronic -not pre-filled or partially pre-filled-PIT,Electronic - fully pre-filled confirmation required-PIT,Other-PIT,Paper returns,Electronic - fully pre-filled deemed,Electronic -not pre-filled or partially pre-filled,Electronic - fully pre-filled confirmation required,Other
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64

Chainning filters before pivoting

In [ ]:
nx.filter_pivot(
    nexus.filter(
        (pl.col("source") == "ISORA") & 
        (pl.col("year") == 2020) 
    ),
    index_cols=INDEX_COLS,
    ind='label'
)

country,year,Capital expenditure - Derived,Operating expenditure - Derived,Salary expenditure - Derived,Information and communications technology expenditure - Derived,FTEs by function of the tax administration-Other functions,Percentage of staff working on headquarter functions,Total tax administration FTEs - Derived,"FTEs by function of the tax administration-Registration, taxpayer services, returns and payment processing",FTEs by function of the tax administration-Enforced debt collection and related functions,"FTEs by function of the tax administration-Audit, investigation and other verification",Operational ICT solutions of the administration are…-Custom built,Operational ICT solutions of the administration are…-On premises commercial off the shelf (COTS),"Operational ICT solutions of the administration are…-Software-as-a-Service (SaaS, i.e. cloud based)",Staff strength levels -No. at end of FY,Staff strength levels -No. at start of FY,Staff strength levels -Departures in FY,Staff strength levels -Recruitments in FY,Academic qualifications (No. of staff at the end of FY)-Masters degree (or above),Academic qualifications (No. of staff at the end of FY)-Bachelors degree,Age distribution (No. of staff at the end of FY)-35-44 years,Age distribution (No. of staff at the end of FY)-45-54 years,Age distribution (No. of staff at the end of FY)-25-34 years,Age distribution (No. of staff at the end of FY)-Under 25 years,Age distribution (No. of staff at the end of FY)-Over 64 years,Age distribution (No. of staff at the end of FY)-55-64 years,Length of service (No. of staff at the end of FY)-Over 19 years,Length of service (No. of staff at the end of FY)-Under 5 years,Length of service (No. of staff at the end of FY)-5-9 years,Length of service (No. of staff at the end of FY)-10-19 years,Gender distribution (No. of staff at the end of FY)-Executives only-Female,Gender distribution (No. of staff at the end of FY)-Executives only-Other,Gender distribution (No. of staff at the end of FY)-All staff-Male,Gender distribution (No. of staff at the end of FY)-All staff-Female,Gender distribution (No. of staff at the end of FY)-Executives only-Male,Gender distribution (No. of staff at the end of FY)-All staff-Other,…,Taxpayers must first pursue internal review where an internal review is permissible,Mechanisms available for taxpayers to challenge assessments-Independent review by external body,FTEs in LTO/P as percentage of FTEs,Percentage of net revenue administered under LTO/P in relation to total net revenue collected by the tax administration,"FTEs on audit, investigation and other verification function in the LTO/P as percentage of total FTEs in LTO/P",Corporate taxpayers managed through LTO/P as percentage of active corporate taxpayers,Total value of additional assessments raised through LTO/P as percentage of total value of additional assessments raised from audits,Corporate taxpayers per FTE in LTO/P,Active taxpayers on PIT register as percentage of Labor Force,Active taxpayers on PIT register as percentage of Population,On PIT register,On CIT register,On PAYE register,On VAT register,On Excise register,On-time filing rate % - Corporate income tax,On-time filing rate % - Personal income tax,On-time filing rate % - Value added tax,On-time filing rate % - Employers that withhold tax from employees,Percent CIT returns e-filed,Percent PIT returns e-filed,Percent VAT returns e-filed,Paper returns-CIT,Electronic - fully pre-filled deemed-CIT,Electronic -not pre-filled or partially pre-filled-CIT,Electronic - fully pre-filled confirmation required-CIT,Other-CIT,Paper returns-PIT,Electronic - fully pre-filled deemed-PIT,Electronic -not pre-filled or partially pre-filled-PIT,Electronic - fully pre-filled confirmation required-PIT,Other-PIT,Paper returns,Electronic - fully pre-filled deemed,Electronic -not pre-filled or partially pre-filled,Electronic - fully pre-filled confirmation required,Other
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64

filter for specific countries

In [12]:
nx.filter_pivot(
    nexus.filter(
        (pl.col("source") == "World Bank") & 
        (pl.col("region_name") == "Africa")
    ),
    index_cols=INDEX_COLS,
    ind='code'  # Use indicator codes instead of labels
)

country,year,WB.PEFA.PI-2011-01,WB.PEFA.PI-2011-02,WB.PEFA.PI-2011-02i,WB.PEFA.PI-2011-03,WB.PEFA.PI-2011-04,WB.PEFA.PI-2011-04i,WB.PEFA.PI-2011-04ii,WB.PEFA.PI-2011-05,WB.PEFA.PI-2011-06,WB.PEFA.PI-2011-07,WB.PEFA.PI-2011-07i,WB.PEFA.PI-2011-07ii,WB.PEFA.PI-2011-08,WB.PEFA.PI-2011-08i,WB.PEFA.PI-2011-08ii,WB.PEFA.PI-2011-08iii,WB.PEFA.PI-2011-09,WB.PEFA.PI-2011-09i,WB.PEFA.PI-2011-09ii,WB.PEFA.PI-2011-10,WB.PEFA.PI-2011-11,WB.PEFA.PI-2011-11i,WB.PEFA.PI-2011-11ii,WB.PEFA.PI-2011-11iii,WB.PEFA.PI-2011-12,WB.PEFA.PI-2011-12i,WB.PEFA.PI-2011-12ii,WB.PEFA.PI-2011-12iii,WB.PEFA.PI-2011-12iv,WB.PEFA.PI-2011-13,WB.PEFA.PI-2011-13i,WB.PEFA.PI-2011-13ii,WB.PEFA.PI-2011-13iii,WB.PEFA.PI-2011-14,WB.PEFA.PI-2011-14i,…,rLabor,rLaborAll,rIndTax,mTaxes,mIT,mPIT,mCIT,mVAT,mExcise,mTrade,mProperty,mSC,tVAT,tExcise,tPIT,tCIT,tProperty,FX.OWN.TOTL.OL.ZS,FX.OWN.TOTL.YG.ZS,FB.BNK.CAPA.ZS,FD.RES.LIQU.AS.ZS,CC.EST,IQ.CPA.PUBS.XQ,IQ.CPA.PADM.XQ,DT.DOD.DSTC.CD,NY.GDP.MKTP.CD,BN.RES.INCL.CD,GC.TAX.TOTL.GD.ZS,DT.NFL.BOND.CD,CM.MKT.LCAP.CD,FS.AST.DOMS.GD.ZS,cc,ge,pv,rl,rq,va
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Zambia""",2005,2.0,1.0,1.0,4.0,1.5,1.0,3.0,2.0,3.0,1.5,1.0,2.0,1.5,2.0,1.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,4.0,1.0,2.5,2.0,4.0,2.0,2.0,3.0,3.0,3.0,3.0,2.5,2.0,…,null,null,17.5,null,null,null,null,null,null,null,null,null,4.391048,2.06511,5.291844,1.204647,0.0,null,null,null,39.327075,-0.59244,3.2,3.0,2.34348e8,8.3319e9,-2.0862e9,14.794769,null,2.5167e9,null,-0.59244,-0.936883,0.108463,-0.517538,-0.742619,-0.430346
"""Gabon""",2006,3.0,1.0,1.0,3.0,2.0,null,null,2.0,2.0,2.0,null,null,2.0,null,null,null,null,null,null,2.0,2.0,null,null,null,1.0,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,1.476613,null,null,null,null,null,null,null,25.542581,-1.083068,null,null,4.2727e8,1.0328e10,null,null,null,null,null,-1.083068,-0.835813,0.220006,-0.754506,-0.301131,-0.950213
"""Ghana""",2006,3.0,1.0,1.0,4.0,3.5,4.0,3.0,3.0,2.0,4.0,4.0,4.0,2.0,4.0,1.0,1.0,2.0,2.0,2.0,3.0,3.0,4.0,3.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,3.0,2.0,2.0,…,null,null,null,null,null,null,null,null,null,null,null,null,2.744267,1.806188,1.310023,1.138426,null,null,null,null,19.761361,-0.014182,3.9,3.5,1.1817e9,2.0885e10,4.1845e8,12.534619,null,7.6e8,null,-0.014182,0.027578,0.006396,0.056497,-0.107714,0.407593
"""Guinea-Bissau""",2006,1.0,1.0,1.0,2.0,1.0,null,null,2.0,2.0,2.0,null,null,null,null,null,null,1.0,null,null,1.0,2.5,null,null,null,1.5,null,null,null,null,2.0,null,null,null,1.5,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,27.529274,-1.095483,2.6,2.5,1.2630e8,6.3478e8,-5.5728e7,null,null,null,null,-1.095483,-1.131372,-0.494073,-1.256102,-0.980062,-0.628868
"""Kenya""",2006,2.0,4.0,4.0,2.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,3.0,4.0,4.0,1.0,2.5,2.0,4.0,3.0,3.0,4.0,4.0,1.0,2.0,2.0,2.0,3.0,1.0,3.0,3.0,3.0,3.0,2.5,2.0,…,null,null,null,null,null,null,null,null,null,null,null,null,2.660621,1.549297,2.008959,1.798354,0.000352,null,null,10.693964,10.066889,-0.935451,3.4,3.5,5.8989e8,2.5826e10,5.8130e8,null,null,1.1378e10,null,-0.935451,-0.65546,-1.126983,-0.882365,-0.239397,-0.130347
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""South Sudan""",2003,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""So